In [2]:
#Correr solo si se va a plotear spectrogramas

import librosa

def plot_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", interpolation="nearest")

In [3]:
import os
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary



In [31]:
#librerias locales
#from ..libs import dataLoaders as mi_dataLoader
#from ..libs import clasificador_padre as mi_clasificador
from importlib.machinery import SourceFileLoader
mi_dataLoader = SourceFileLoader('dataLoadersV2', '../libs/dataLoadersV2.py').load_module()
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

# Cargando un DataSet

In [17]:
#Busqueda del path para el data set 

dataSetName = 'onlywoman'
root = './'
while not os.path.isdir(f'{root}data'):
    root += '../'
Path_train = root+'data/'+dataSetName+'_train.csv'
Path_val = root+'data/'+dataSetName+'_val.csv'
Path_test = root+'data/'+dataSetName+'_test.csv'

In [6]:

testDf = pd.read_csv(Path_train)
#testDf.keys()
testDf = testDf.sample(frac=1, random_state = 98 )[0:200] #separo 200 muestras
testDataSet = mi_dataLoader.clasification_dataset(testDf, file_col="fileMfcc", target_col="artist", seed = 98)


100%|██████████| 200/200 [00:00<00:00, 673.76it/s]


# Generando el DataLoader

In [7]:
test_dataloader = DataLoader(testDataSet, batch_size=100, shuffle=True)
#muestra_data, muestra_target = next(iter(test_dataloader))
#muestra_target

In [8]:
###Muestras de especrogramas

#train_features, train_labels = next(iter(test_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
#plot_spectrogram(train_features[0], train_labels[0])
#img = train_features[0].squeeze()
#label = train_labels[0]
#plt.imshow(img, cmap="gray")
#plt.show()
#print(f"Label: {label}")

# instanciando un modelo

In [67]:
from importlib.machinery import SourceFileLoader
mi_dataLoader = SourceFileLoader('dataLoadersV2', '../libs/dataLoadersV2.py').load_module()
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

os.listdir('./test/state')
pState = "./test/state/"
pHist = "./test/history/"
overWrite = False
show_metrics = True
sLoad = 10 #save state a cargar
lr = 0.001
constructor = mi_clasificador.onlyWoman_MFCC_02_N2

In [68]:
def test_accur (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)


if((len(os.listdir(pState)) == 0) or overWrite):
    history = {}
    history['loss'] = torch.empty(0)
    history['acur'] = torch.empty(0)

    clasificador = constructor(list(testDataSet.dictionary.keys()))
else:
    history =  torch.load(pHist+os.listdir(pHist)[sLoad])

    save_state = torch.load(pState+os.listdir(pState)[sLoad])
    clasificador = constructor(list(testDataSet.dictionary.keys()))
    clasificador.load_state_dict(save_state)

test_optimizer = torch.optim.Adam(clasificador.parameters(), lr=lr)
test_criterion = torch.nn.CrossEntropyLoss()
print(clasificador.modules)


C:\Users\Alenca\AppData\Local\Temp\ipykernel_7188\3912412090.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  history =  torch.load(pHist+os.listdir(pHist)[sLoad])
C:\Us

<bound method Module.modules of onlyWoman_MFCC_02_N2(
  (inPut): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (chanelUp): Sequential(
    (0): Conv2d(4, 16, kernel_size=(1, 7), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(1, 7), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(1, 7), stride=(1, 2))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (justTime): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (out): Sequential(
    (0): 

# Entrenando el modelo

In [58]:
%matplotlib qt5

if show_metrics:
    plt.ion()
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize = (6,6))
    l1, = axes[0].plot([], [])
    l2, = axes[1].plot([], [])
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Acur')

    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')

for i in tqdm(range(15)):
    loss_log, acur_log = clasificador.train_loop(1, test_dataloader, test_optimizer , test_criterion, test_accur)
    history['loss'] = torch.cat((history['loss'], loss_log), dim=0)
    history['acur'] = torch.cat((history['acur'], acur_log), dim=0)
    save_state = clasificador.state_dict()
    n = len(os.listdir(pState))
    torch.save(save_state, f"{pState}/ss_{n}.pt")
    torch.save(history, f"{pHist}/hist_{n}.pt")
    
    if show_metrics:
        # Grafica
        l1.set_data(range(len(history['acur'])), history['acur'])
        axes[0].relim()  # Recalculate data limits
        axes[0].autoscale_view()  # Rescale axes

        l2.set_data(range(len(history['loss'])), history['loss'])
        axes[1].relim()  # Recalculate data limits
        axes[1].autoscale_view()  # Rescale axes
        
        fig.canvas.draw()  # Redraw the figure
        fig.canvas.flush_events()  # Ensure events are processed
        plt.pause(0.5)

if show_metrics:
    plt.ioff()
    plt.show()

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 0 of 1 / Loss: 1.089447259902954 / Accuracy: 0.3700000047683716


  7%|▋         | 1/15 [01:06<15:31, 66.57s/it]

Epoch 0 of 1 / Loss: 1.0497747659683228 / Accuracy: 0.38499999046325684


 13%|█▎        | 2/15 [01:54<12:05, 55.82s/it]

...

 87%|████████▋ | 13/15 [09:42<01:22, 41.30s/it]

Epoch 0 of 1 / Loss: 0.6936148405075073 / Accuracy: 0.9900000095367432


 93%|█████████▎| 14/15 [10:23<00:41, 41.25s/it]

Epoch 0 of 1 / Loss: 0.6784189939498901 / Accuracy: 1.0


100%|██████████| 15/15 [11:04<00:00, 44.28s/it]


In [52]:
plt.figure()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize = (8,8))

# Graficar en el primer subplot
axes[0].plot(range(len(history['acur'])), history['acur'], 'r-', label='Acuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Acur')
axes[0].legend()

axes[1].plot(range(len(history['loss'])), history['loss'], 'r-', label='Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

plt.show()

# Validando

In [69]:
#Carga de las muestras y el DataLoader
valDf = pd.read_csv(Path_val)
valDataSet = mi_dataLoader.clasification_dataset(valDf, file_col="fileMfcc", target_col="artist", seed = 98)
val_dataloader = DataLoader(valDataSet, batch_size=100, shuffle=False)


# Cargo el modelo
#save_state = torch.load("./modelos/goliat/goliat_0_03.pt")
#clasificador = mi_clasificador.Clasificador_03(list(valDataSet.dictionary.keys()))
#clasificador.load_state_dict(save_state)

  0%|          | 0/2640 [00:00<?, ?it/s]

100%|██████████| 2640/2640 [00:04<00:00, 648.45it/s]


In [70]:
def acuracy_fn (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)

loss_fn = torch.nn.CrossEntropyLoss()

#Separando las muestras y los targets
muestras = []
targets = []
for muestra_data, muestra_target in val_dataloader:
    muestras.append(muestra_data)
    targets.append(torch.tensor(muestra_target))
    #print(len(muestras)*100,"/",len(valDataSet))
#muestras[0].shape



C:\Users\Alenca\AppData\Local\Temp\ipykernel_7188\716157085.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets.append(torch.tensor(muestra_target))


In [71]:
resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn)
print(resultado_evalucion) #batch norm post chanel up y post extract

100%|██████████| 27/27 [02:19<00:00,  5.17s/it]

(1.0804423380781103, 0.35814814814814816)


In [ ]:
resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn)
print(resultado_evalucion) #batch norm post chanel up

100%|██████████| 27/27 [02:20<00:00,  5.20s/it]

(0.9133836185490644, 0.6470370370370371)


In [ ]:
resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn)
print(resultado_evalucion)

100%|██████████| 27/27 [02:58<00:00,  6.60s/it]

(0.9401003078178123, 0.6066666666666668)


## Codigo usado mientras se generaba la arquitectura (osea, codigo desechable)

In [ ]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

In [24]:
muestra_data, muestra_target = next(iter(test_dataloader))
#muestra = torch.stack(muestra, 0)
muestra_data.shape

torch.Size([100, 13, 997])

In [59]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()
clasificador = mi_clasificador.onlyWoman_MFCC_01(list(testDataSet.dictionary.keys()))

In [60]:
res =clasificador(muestra_data)
res.shape

In: torch.Size([100, 13, 997])
squeeze: torch.Size([100, 1, 13, 997])
Shape post 1D:  torch.Size([100, 4, 15, 999])
post flatten: torch.Size([100, 136136])
Alive


torch.Size([100, 3])

In [45]:
out = nn.Sequential(   # bloque que acondiciona la salida
    nn.Linear(*self.architecture[-1]), #Lineal que sale a las clases objetivo
    nn.Softmax( dim = 1))

NameError: name 'self' is not defined

In [61]:
test_criterion(res,muestra_target)

tensor(1.1800, grad_fn=<DivBackward1>)

In [62]:
test_accur (res, muestra_target)

0.3

In [64]:
#importlib.reload(mi_clasificador)
clasificador = mi_clasificador.Clasificador_01(testDataSet.dictionary.keys())
res =clasificador(muestra_data)
test_accur (res, muestra_target)


TypeError: _Clasificador_padre.__init__() missing 1 required positional argument: 'classes'

In [318]:
(torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)).sum().item() / len(muestra_target)

0.0

In [319]:
torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)

tensor([False])

In [42]:
muestra_target.shape, res.shape

(torch.Size([3, 3]), torch.Size([3, 3]))

In [43]:
torch.argmax(res, axis = 1)

tensor([0, 2, 2])

In [44]:
res

tensor([[9.9844e-01, 2.4358e-21, 1.5574e-03],
        [1.7411e-05, 4.4522e-07, 9.9998e-01],
        [0.0000e+00, 0.0000e+00, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [43]:
torch.argmax(muestra_target, axis=1)

tensor([0, 0, 0, 0, 0])

In [44]:
muestra_target

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [21]:
muestra_target == res

tensor([[False,  True, False],
        [False,  True, False],
        [False, False, False],
        [False,  True, False],
        [False, False, False]])

In [45]:
res

tensor([[0.0000e+00, 4.1355e-15, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00, 4.7325e-08],
        [0.0000e+00, 1.0000e+00, 4.9665e-37],
        [1.1335e-33, 3.0150e-14, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [ ]:
flat = nn.Flatten(start_dim = 0)
lin = nn.Linear(17920,3)
soft = nn.Softmax(dim = 0)

print("Red: ",res.shape)
aux = flat(res)
print("Flat: ",aux.shape)
aux = lin(aux)
print("Line: ",aux.shape)
aux = soft(aux)
print("Soft:",aux.shape)

Red:  torch.Size([128, 5, 28])
Flat:  torch.Size([17920])
Line:  torch.Size([3])
Soft: torch.Size([3])
